In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
sns.set_theme(style="ticks", color_codes=True)
from tqdm.notebook import tqdm, trange
import time
from sklearn.metrics import roc_auc_score, accuracy_score


### 2022/07~2022/12檔案讀取 

In [9]:
df = pd.read_csv('../../../../homecredit/train_all_v1.csv')
#'../files/dfTime/year/time_all.parquet'
print(df.shape)
df

(307511, 63)


,Unnamed: 0,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,creditcard_low,instalments_overdue_DAY_mean,instalments_overdue_AMT_mean,instalments_PERIOD,BUREAU_DAYS_CREDIT,BUREAU_DAYS_CREDIT_ENDDATE,BUREAU_DAYS_ENDDATE_FACT,CDCS_ratio,bureau_annuity_mean,bereau_balance_status
0,0,100002,1,M,N,Y,0,202500.0,406597.5,24700.5,...,0.0,-20.421053,0.000000,19.000000,-874.000000,-134.064824,-777.474819,0.348965,100580.0,0.401557
1,1,100003,0,F,N,N,0,270000.0,1293502.5,35698.5,...,0.0,-7.160000,0.000000,8.333333,-1400.750000,-544.500000,-1077.349819,0.000000,70437.0,0.000000
2,2,100004,0,M,Y,Y,0,67500.0,135000.0,6750.0,...,0.0,-7.666667,0.000000,3.000000,-867.000000,-488.500000,-532.500000,0.000000,54004.5,0.000000
3,3,100006,0,F,N,Y,0,135000.0,312682.5,29686.5,...,0.0,-19.375000,0.000000,5.333333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,4,100007,0,M,N,Y,0,121500.0,513000.0,21865.5,...,0.0,-3.636364,-452.384318,12.600000,-1149.000000,-783.000000,-783.000000,0.000000,16645.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,307506,456251,0,M,N,N,0,157500.0,254700.0,27558.0,...,0.0,-36.285714,0.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
307507,307507,456252,0,F,N,Y,0,72000.0,269550.0,12001.5,...,0.0,-2.833333,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
307508,307508,456253,0,F,N,Y,0,153000.0,677664.0,29979.0,...,0.0,-14.500000,-283.792500,7.333333,-867.500000,280.500000,-905.699639,0.453493,11269.0,1.000000
307509,307509,456254,1,F,N,Y,0,171000.0,370107.0,20205.0,...,0.0,-19.000000,0.000000,9.500000,-1104.000000,-859.000000,-859.000000,0.000000,7737.0,1.000000


In [ ]:
df.isnull().sum()

#### 不做獨熱編碼 

In [ ]:
df = pd.get_dummies(df, columns=['Name'], prefix='Name',dtype=int)

#### 將 is_holiday轉成0、1

In [ ]:
df['is_holiday'] = df['is_holiday'].astype(int)
#df =df.drop(['PULocationID'],axis=1)
df

#### 做讀熱不將 Name轉成1、2、3

In [ ]:
df['Name'].replace({'lyft': 2, 'uber': 3, 'yellow': 1}, inplace=True)
df

### 不加特徵不刪除 

In [ ]:
df = df.drop(['PULocationID','Nhp','Name_yellow'],axis=1)
df

In [ ]:
train_data=df[df['month'] < 11]
train_data

In [ ]:
test_data = df[df['month'] > 10]
test_data

In [ ]:
df1 = pd.read_parquet('../files/dfTime/year/TS3-2023.parquet')
df1

In [ ]:
df1['Name'].replace({'lyft': 2, 'uber': 3, 'yellow': 1}, inplace=True)
df1['is_holiday'] = df1['is_holiday'].astype(int)
#df1 = pd.get_dummies(df1, columns=['Name'], prefix='Name',dtype=int)
#df1 =df1.drop(['PULocationID'],axis=1)
df1

In [ ]:
val_data=df1
val_data

In [ ]:
del df,df1

參考北捷隨機森林，把上車站轉換成平均每日總數，
因各車種載客數差距過大，故以車種跟地區ID當作加總的依據

將資料分為訓練集(60%)
         驗證集(20%)
         測試集(20%)

### 測試模型 

In [11]:
import datetime
from sklearn.preprocessing import *
import re
start_time = time.time()

obj_columns = df .select_dtypes(include=['object']).columns
# Apply one-hot encoding to object columns
df  = pd.get_dummies(df , columns = obj_columns, drop_first = True)

X= df.drop(columns= ['Unnamed: 0','TARGET','SK_ID_CURR']).rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
y = df.TARGET


X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=37)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=37)





end_time = time.time()  
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())    



程式執行花費的時間： 0.88 秒
done時間: 2023-09-01 11:20:53.345863


In [12]:
start_time = time.time()

X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)
X_val = np.array(X_val)
y_val = np.array(y_val)

end_time = time.time()  
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())  



程式執行花費的時間： 0.18 秒
done時間: 2023-09-01 11:20:56.068624


In [13]:
start_time = time.time()

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)
X_val_scaled = scaler.fit_transform(X_val)

end_time = time.time()  
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now()) 



程式執行花費的時間： 0.32 秒
done時間: 2023-09-01 11:21:00.751607


In [18]:
print("start時間:", datetime.datetime.now())
print('------------------') 
from sklearn.ensemble import RandomForestClassifier

start_time = time.time()

rforest = RandomForestClassifier(n_estimators = 100, min_samples_leaf=5, random_state = 0, n_jobs=-1,max_depth=50)
rforest.fit(X_train, y_train)
end_time = time.time()  
execution_time = end_time - start_time
print("\n程式執行花費的時間：", round(execution_time,2), "秒")
print("done時間:", datetime.datetime.now())


start時間: 2023-09-01 12:46:03.211386
------------------

程式執行花費的時間： 21.13 秒
done時間: 2023-09-01 12:46:24.338021


In [15]:
# print("start時間:", datetime.datetime.now())
# print('------------------') 
# start_time = time.time()
# predictions = rforest.predict(X_val)

# #R Square
# import statsmodels.api as sm
# X_addC = sm.add_constant(predictions)
# result = sm.OLS(y_val, X_addC).fit()
# print(result.rsquared, result.rsquared_adj)
# print('------------------')
# end_time = time.time()  
# execution_time = end_time - start_time
# print("\n程式執行花費的時間：", round(execution_time,2), "秒")
# print("done時間:", datetime.datetime.now())

start時間: 2023-09-01 12:34:40.401830
------------------
0.0802866428145832 0.08022682016874061
------------------

程式執行花費的時間： 2.1 秒
done時間: 2023-09-01 12:34:42.503597


In [ ]:
del rforest

In [20]:


import datetime
import time
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
y_pred_proba = rforest.predict_proba(X_test)[:, 1]  # 获取正类的预测概率
y_pred = rforest.predict(X_test)  # 获取预测的类别

auc = roc_auc_score(y_test, y_pred_proba)
acc = accuracy_score(y_test, y_pred)

print("AUC:", auc)
print("Accuracy:", acc)
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("R-squared:", r2)
print("Mean Squared Error:", mse)




AUC: 0.7361436974881803
Accuracy: 0.921436004162331
R-squared: -0.0860850463949121
Mean Squared Error: 0.0785639958376691
